In [119]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import pickle
import pylab as pl # R2?

# Load the Boston housing dataset
boston = pd.read_csv('housing.csv')
features = boston.drop('house_value' , 1).values
y = boston['house_value'].values





# Split the data into test and training sets
X_train, X_test, y_train, y_test = train_test_split(features, y, test_size=0.25, random_state=1978)

lm = linear_model.LinearRegression()
model = lm.fit(X_train,y_train)


# R2 training
print("Residual sum of squares (train set): %.2f"
      % lm.score(X_train,y_train))
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lm.predict(X_train) - y_train) ** 2))

print("Residual sum of squares: %.2f"
      % mean_squared_error(y_train, lm.predict(X_train)))

n = X_train.shape[0]
m = X_train.shape[1]

# sum square
s = np.sum((lm.predict(X_train) - y_train) ** 2)/(n-(m-1)-1) 
print(s)

# standard deviation, square root of the diagonal of variance-co-variance matrix (sigular vector decomposition)
# aka var_est
sd_alpha = np.sqrt(s*(np.diag(np.linalg.pinv(np.dot(X_train.T,X_train))))) 

#print(type(sd_alpha
#    ))
#print("standard deviation: %.2f"
#      % np.sqrt(s*(np.diag(np.linalg.pinv(np.dot(X_train.T,X_train))))) )

SE_est = np.sqrt(sd_alpha)
#print(SE_est)

#print("RSTD DEV: %.2f"
#      % SE_est)

# Test set:

# R2
print("R2 (test set): %.2f"
      % lm.score(X_test,y_test))

# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((lm.predict(X_test) - y_test) ** 2))

print("Residual sum of squares: %.2f"
      % mean_squared_error(y_test, lm.predict(X_test)))


class LRPI:
    def __init__(self, normalize=False, n_jobs=1, t_value = 2.13144955): #  t value to get the 95% conf. interval.
        self.normalize = normalize
        self.n_jobs = n_jobs
        self.LR = linear_model.LinearRegression(normalize=self.normalize, n_jobs= self.n_jobs)
        self.t_value = t_value
        
    def fit(self, X_train, y_train):
        self.X_train = pd.DataFrame(X_train)
        self.y_train = pd.DataFrame(y_train)
        
        self.LR.fit(self.X_train, self.y_train)
        X_train_fit = self.LR.predict(self.X_train)
        self.MSE = np.power(self.y_train.subtract(X_train_fit), 2).sum(axis=0) / (self.X_train.shape[0] - self.X_train.shape[1] - 1)
        self.X_train.loc[:, 'const_one'] = 1
        self.XTX_inv = np.linalg.inv(np.dot(np.transpose(self.X_train.values) , self.X_train.values))
    
    def predict(self, X_test):
        self.X_test = pd.DataFrame(X_test)
        self.pred = self.LR.predict(self.X_test)
        self.X_test.loc[: , 'const_one'] =1
        SE = [np.dot(np.transpose(self.X_test.values[i]) , np.dot(self.XTX_inv, self.X_test.values[i]) ) for i in range(len(self.X_test)) ]
        results = pd.DataFrame(self.pred , columns=['Pred'])
        
        results.loc[:,"lower"] = results['Pred'].subtract((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        results.loc[:,"upper"] = results['Pred'].add((self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) )),  axis=0)
        results.loc[:,"duu"] = (self.t_value)* (np.sqrt(self.MSE.values + np.multiply(SE,self.MSE.values) ))
        results.loc[:,"se"] = SE
            
        return results
    
    
model = LRPI()
model.fit(X_train, y_train)
# save the model to disk
filename = 'lm_model.sav'
#pickle.dump(model, open(filename, 'wb'))

#model = pickle.load(open(filename, 'rb'))

a = X_test[1,]
b = a.reshape(-1,len(a))

results = model.predict(X_test[1:2,])
#print(b) nää on samat
#print(X_test[1:2,])
print(results.head(10))

# one-two-tree standard deviation

print(np.std(y_test))

## add a column of ones for the constant intercept term
#X = np.vstack(( X_test, np.ones( X_test.size ) ))

## convert the NumPy array to matrix
#X = np.matrix( X )
 
## perform the matrix multiplication,
## and then take the inverse
#C = np.linalg.inv( X * X.T )
 
## multiply by the MSE of the residuals
#C *= result.mse_resid
 
## take the square root
#SE = np.sqrt(C)
 
#print SE



## Random Forest


#rf = RandomForestRegressor(n_estimators=1000, min_samples_leaf=1)
#rf.fit(X_train, y_train)
# save the model to disk
filename = 'rf_model.sav'
#pickle.dump(rf, open(filename, 'wb'))

rf = pickle.load(open(filename, 'rb'))
print("RF test set MSE: %.2f"
      % np.mean((y_test - rf.predict(X_test))**2)) # MSE

def pred_ints(model, X, percentile=95):
    err_down = []
    err_up = []
    preds = []
    for pred in model.estimators_:
        preds.append(pred.predict(X)[0])
    err_down.append(np.percentile(preds, (100 - percentile) / 2. ))
    err_up.append(np.percentile(preds, 100 - (100 - percentile) / 2.))
    return err_down, err_up

a = X_test[1,]
b = a.reshape(-1,len(a))


err_down, err_up = pred_ints(rf, b)
pred = rf.predict(b)
 
print(err_down, pred, err_up)

#truth = Y[idx[trainsize:]]
#correct = 0.
#for i, val in enumerate(truth):
#    if err_down[i] <= val <= err_up[i]:
#        correct += 1
#print correct/len(truth)



#sd_alpha = np.sqrt(s*(np.diag(np.linalg.pinv(np.dot(X.T,X))))) 
#print(sd_alpha)

#predictions = lm.predict(X_test)

#yb = boston.target.reshape(-1, 1)
#Xb = boston['data'][:,5].reshape(-1, 1)


# Create linear regression object
#regr = linear_model.LinearRegression()
# Train the model using the training sets
#regr.fit( Xb, yb)

Residual sum of squares (train set): 0.62
Residual sum of squares: 32.35
Residual sum of squares: 32.35
32.7800580232
R2 (test set): 0.56
Residual sum of squares: 36.85
Residual sum of squares: 36.85
        Pred      lower      upper        duu        se
0  25.006331  12.676546  37.336117  12.329785  0.018088
9.1089376299
RF test set MSE: 32.87
[22.0] [ 26.0136] [32.0]
